In [3]:
import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>
    div.output_area{
        max-height:10000px;
        overflow:scroll;
    }
</style>"""))

Notebook Last Run Initiated: 2018-09-20 11:30:26.394143


In [4]:
%load_ext autoreload
%autoreload
from datetime import datetime
import numpy as np
import pandas as pd
pd.set_option("display.max_rows",1000)
pd.set_option("display.max_columns",200)
import warnings
warnings.filterwarnings('ignore')
import math, random

import matplotlib
import matplotlib.pyplot as plt
from os.path import exists,join
import seaborn as sns
from scipy.stats import kendalltau
from haversine import haversine
import pygeohash as geohash
import sys

from modelio import saveData, loadData, saveSparkData, appendSparkData, saveJoblib, loadJoblib
from timeUtils import clock, elapsed, getTimeSuffix, getDateTime, addDays, printDateTime, getFirstLastDay


clock()
print(sys.version)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Thu Sep 20, 2018 11:30:26 for Begin
3.6.5 |Anaconda, Inc.| (default, Apr 26 2018, 08:42:37) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]


In [5]:
def getCities():
    cities={}
    cities['naperville'] = {"lat": 41.750839, "long": -88.153535, "radius": 0.05}
    cities['knoxville']  = {'lat': 35.964668, 'long': -83.926453, 'radius': 0.15}
    cities['evanston']   = {'lat': 42.041166, 'long': -87.690163, 'radius': 0.05}
    cities['wrigley']    = {'lat': 41.948437, 'long': -87.655334, 'radius': 0.03} # Wrigley Field
    cities['alexandria'] = {'lat': 38.80472,  'long': -77.04722, 'radius': 0.075}  # Alexandria, VA
    cities['dallas']     = {'long': -97.04044, 'lat': 32.897480, 'radius': 0.15}
    cities['urbana']     = {'long': -88.24338, 'lat': 40.116421, 'radius': 0.15}
    cities['portjeff']   = {'long': -73.06927, 'lat': 40.946487, 'radius': 0.15}
    cities['r4h']        = {'lat': 38.88184,  'long': -77.10806, 'radius': 0.05} # r4h
    cities['r4h']        = {'lat': 41.76,  'long': -88.3200715, 'radius': 0.05} # r4h
    return cities

colors = [ 'red', 'blue', 'gray', 'darkred', 'lightred', 'orange', 'beige', 'green', 'darkgreen', 'lightgreen', 'darkblue', 'lightblue', 'purple', 'darkpurple', 'lightgray']
    
def distHash(gcode1, gcode2):
    """
    distHash(gcode1, gcode2)

    inputs: gcode1 (geohash), gcode2 (geohash)
    outputs: distance (km)
    """
    pnt1 = geohash.decode_exactly(gcode1)[:2]
    pnt2 = geohash.decode_exactly(gcode2)[:2]
    dist = haversine(pnt1, pnt2)
    return dist

def getPOIByCity(cityname):
    poi = loadJoblib('poi-7.p')
    cities = getCities()
    for city,citydata in cities.items():
        if city != cityname:
            continue
        print("City --> {0}".format(city))
        geocity = geohash.encode(citydata['lat'], citydata['long'], precision=7)
        poicity = {}
        for k,v in poi.items():
            poicity[k] = set()
            for geo in v:
                try:
                    dist = distHash(geocity, geo)
                except:
                    continue
                if dist < citydata['radius']*111:
                    poicity[k].add(geo)
            print("  Keeping {0} out of {1}".format(len(poicity[k]), len(v)))
        print("--> {0}".format(city))
        saveJoblib(poicity, 'poi-{0}.p'.format(city))
#getPOIByCity('r4h')

city='evanston'

In [6]:
def getBoundary(points):    
    from scipy.spatial import ConvexHull
    hull     = ConvexHull(points)
    boundary = [points[x] for x in hull.vertices]
    return boundary

# Skim Geomaps

In [52]:
prec=7
from glob import glob
from os.path import join, basename
files = glob("geomap-{0}-*.p".format(prec))

import folium
cities = getCities()
pnt2=(cities[city]['lat'], cities[city]['long'])

from os import mkdir
savedir = "geoskims/{0}".format(city)
try:
    mkdir(savedir)
except:
    pass

distMax = 5
import pickle
from modelio import saveJoblib
for ifile in files:
    print("Loading {0}".format(ifile))
    data = pickle.load(open(ifile, "rb"))
    print("  --> Found {0} geos".format(len(data)))
    save = {}
    for geo, rec in data.items():
        lat,long = geohash.decode_exactly(geo)[:2]
        pnt1 = (lat, long)
        dist = haversine(pnt1, pnt2)
        if dist > distMax:
            continue
        save[geo] = rec
    if len(save) > 0:
        print("  -->  Keep {0} geos".format(len(save)))    
        savename = join(savedir, basename(ifile))
        #print("Saving {0} geos to {1}".format(len(save), savename))
        saveJoblib(save, savename)
    #pickle.dump(save, open(savename, "wb"))

Loading geomap-7-HEREPOI.p
  --> Found 186738 geos
  -->  Keep 129 geos
  --> This file is 1.2kB.
Loading geomap-7-HighwayA.p
  --> Found 1311011 geos
Loading geomap-7-HighwayB.p
  --> Found 924506 geos
Loading geomap-7-HighwayC.p
  --> Found 1076021 geos
  -->  Keep 9 geos
  --> This file is 126B.
Loading geomap-7-Industry.p
  --> Found 9587 geos
Loading geomap-7-InterstateA.p
  --> Found 253770 geos
Loading geomap-7-InterstateB.p
  --> Found 152055 geos
Loading geomap-7-InterstateC.p
  --> Found 194921 geos
Loading geomap-7-MajorRdA.p
  --> Found 1754319 geos
Loading geomap-7-MajorRdB.p
  --> Found 1295815 geos
Loading geomap-7-MajorRdC.p
  --> Found 1201250 geos
  -->  Keep 2312 geos
  --> This file is 16.9kB.
Loading geomap-7-pofwOSM.p
  --> Found 257804 geos
  -->  Keep 197 geos
  --> This file is 1.4kB.
Loading geomap-7-poisOSM.p
  --> Found 2828691 geos
  -->  Keep 1253 geos
  --> This file is 10.6kB.
Loading geomap-7-powfOSM.p
  --> Found 0 geos
Loading geomap-7-Rail.p
  --> Fo

# Add To Folium

In [53]:
skimdata = {}
prec=7
from glob import glob
from modelio import loadJoblib
from os.path import join, basename, splitext
from os import remove
files = glob("geoskims/{0}/*.p".format(city))
for ifile in files:
    name = splitext(basename(ifile))[0].split('-')[-1]
    skimdata[name] = loadJoblib(ifile)    
    if len(skimdata[name]) == 0:
        remove(ifile)
        continue
    print(len(skimdata[name]),'\t',name)

2728 	 Car
83 	 College
2183 	 Commerce
129 	 HEREPOI
9 	 HighwayC
1 	 Industry
1406 	 InterstateC
2312 	 MajorRdC
197 	 pofwOSM
1253 	 poisOSM
153 	 Rail
71 	 Schools
3 	 StateRteC
2 	 Terminals
628 	 trafficOSM
406 	 transportOSM
26 	 USRteC
2 	 Venues


In [43]:
colors = [ 'red', 'blue', 'gray', 'darkred', 'lightred', 'orange', 'beige', 'green', 'darkgreen', 'lightgreen', 'darkblue', 'lightblue', 'purple', 'darkpurple', 'pink', 'cadetblue', 'lightgray', 'black' ]

def addData(m, dc, name, geovalues, cmap, radius=75, useBB=False, fill=False, useIcon=False):
    from timeUtils import clock, elapsed
    from modelio import saveJoblib, loadJoblib
    from folium import PolyLine, CircleMarker, Circle, Marker, Icon, FeatureGroup
    start,cmt = clock("Adding to folium")
    
    feature_group = FeatureGroup(name=name)

    for geo,weight in geovalues.items():
        color='gray'
        if isinstance(cmap, dict):
            for val,col in cmap.items():
                if weight.get(val) is not None:
                    color=col
                    break
        else:
            color=cmap
        if useBB:
            points = dc.getBB(geo, istuple=True)
            PolyLine(points, color=color, weight=len(weight), opacity=1, popup=",".join(weight.keys())).add_to(feature_group)
        elif useIcon:
            center = dc.getCenter(geo)
            Marker(center, icon=Icon(color='blue', icon_color='white', icon=col, angle=0, prefix='fa'), popup=",".join(weight.keys())).add_to(feature_group)
        else:
            if fill:
                center = dc.getCenter(geo)
                Circle(center, color=color, radius=radius, fill=True, fill_color=color, weight=len(weight)/2, opacity=1, popup=", ".join(weight.keys())).add_to(feature_group)
            else:
                center = dc.getCenter(geo)
                Circle(center, color=color, radius=radius, weight=len(weight)/2, opacity=1, popup=", ".join(weight.keys())).add_to(feature_group)
                
    
    feature_group.add_to(m)

In [54]:
import folium
from collections import OrderedDict
cities = getCities()
pnt2=(cities[city]['lat'], cities[city]['long'])
m = folium.Map(location=list(pnt2), zoom_start=13)
from geoClusteringPlots import drawClusters
dc = drawClusters()


cmap = OrderedDict({'parking': 'cadetblue', 'fuel': 'orange'})
addData(m, dc, "Traffic", skimdata['trafficOSM'], cmap, useBB=True)

cmap = OrderedDict({'entertainment': 'darkred', 'college': 'darkgreen', 'municipal': 'blue', 'attraction': 'green', 'grocery': 'pink', 'auto': 'beige'})
addData(m, dc, "POI", skimdata['poisOSM'], cmap, radius=75)

cmap = OrderedDict({'railstation': 'darkpurple', 'busstop': 'darkpurple'})
addData(m, dc, "Transport", skimdata['transportOSM'], cmap, radius=60, fill=True)

addData(m, dc, "HERE", skimdata['HEREPOI'], cmap='lightblue', radius=45, fill=True)

addData(m, dc, "Interstate", skimdata['InterstateC'], cmap='darkgreen', useBB=True)

addData(m, dc, "Religious", skimdata['pofwOSM'], cmap='red', radius=30, fill=True)

addData(m, dc, "Terminals", skimdata['Terminals'], cmap='darkred', useBB=True)

folium.LayerControl().add_to(m)
#cmap = OrderedDict({'PublicSchools': 'green', 'PrivateSchools': 'green'})
#addData(m, dc, skimdata['Schools'], cmap, radius=60)



Current Time is Thu Sep 20, 2018 13:29:10 for Adding to folium
Current Time is Thu Sep 20, 2018 13:29:10 for Adding to folium
Current Time is Thu Sep 20, 2018 13:29:10 for Adding to folium
Current Time is Thu Sep 20, 2018 13:29:11 for Adding to folium
Current Time is Thu Sep 20, 2018 13:29:11 for Adding to folium
Current Time is Thu Sep 20, 2018 13:29:11 for Adding to folium
Current Time is Thu Sep 20, 2018 13:29:11 for Adding to folium


In [55]:
m.save("{0}.html".format(city))

In [89]:
#

In [30]:
from pandas import DataFrame
df = DataFrame(skimdata['poisOSM'])

In [32]:
df.index

Index(['artwork', 'bakery', 'bank', 'bar', 'beautyshop', 'bicycleshop',
       'bookshop', 'cafe', 'campsite', 'cardealership', 'carwash', 'chemist',
       'clothes', 'college', 'communitycentre', 'convenience', 'dentist',
       'departmentstore', 'doityourself', 'drinkingwater', 'fastfood',
       'firestation', 'fountain', 'furnitureshop', 'golfcourse', 'graveyard',
       'hairdresser', 'hospital', 'hotel', 'jeweller', 'kindergarten',
       'laundry', 'library', 'mall', 'memorial', 'mobilephoneshop', 'museum',
       'optician', 'outdoorshop', 'park', 'pharmacy', 'pitch', 'playground',
       'police', 'postbox', 'postoffice', 'pub', 'publicbuilding',
       'restaurant', 'school', 'shelter', 'shoeshop', 'sportscentre',
       'sportsshop', 'stadium', 'stationery', 'supermarket', 'swimmingpool',
       'theatre', 'toilet', 'tower', 'townhall', 'toyshop', 'track',
       'veterinary', 'viewpoint', 'watertower'],
      dtype='object')

In [ ]:
fastfood=["cafe", "bakery", ""]
slowfood=["pub", "bar", "restaurant"]
retail=["bakery", "cafe"]
recreation=["campsite", "track", "golfcourse", "swimmingpool", "playground", "stadium", "sportscentre"]
medical=["veterinary", "hospital", "dentist", "optician", "pharmacy"]
municipal=["firestation", "police", "postoffice", "publicbuilding", "communitycentre"]
school=["school", "kindergarten"]